In [ ]:
# do all necessary imports for this chapter
using LinearAlgebra
using TensorOperations
using TensorKit
using KrylovKit
using Printf
using Trapz
using Plots
using LaTeXStrings
include("TutorialFunctions.jl")
using .TutorialFunctions: createMPS, mixedCanonical, minAcC

# [Tangent-space methods for uniform matrix product states](https://doi.org/10.21468/SciPostPhysLectNotes.7)

## 3. Transfer matrices and fixed points

### 3.1 MPS as fixed points of one-dimensional transfer matrices

Matrix product states have been used extensively as a variational ansatz for ground states of local Hamiltonians. In recent years, it has been observed that they can also provide accurate approximations for fixed points of transfer matrices. In this notebook we investigate how tangent-space methods for MPS can be applied to one-dimensional transfer matrices.

A one-dimensional transfer matrix in the form of a *matrix product operator* (MPO) is given by

$$
T(O) = \sum_{\{i\}, \{j\}} \left( \dots O^{i_{n-1}, j_{n-1}} O^{i_{n}, j_{n}} O^{i_{n+1}, j_{n+1}} \dots \right)
\left | i_{n-1} \right \rangle \left \langle j_{n-1} \right | \otimes \left | i_{n} \right \rangle \left \langle j_{n} \right | \otimes \left | i_{n+1} \right \rangle \left \langle j_{n+1} \right | \dots \;,
$$

which can be represented diagrammatically as

<center><img src="./img/transferMpo.svg" alt="transferMpo"/></center>

Such an object naturally arises in the context of infinite two-dimensional tensor networks, which can be interpreted as an infinite power of a corresponding one-dimensional row-to-row transfer matrix. This means that the contraction of the network is equivalent to finding the leading eigenvector $\left | \Psi \right \rangle$, referred to as the *fixed point*, of the transfer matrix which satisfies the equation

$$T(O) \left | \Psi \right \rangle \propto \left | \Psi \right \rangle.$$

We can now propose an MPS ansatz for this fixed point, such that it obeys the eigenvalue equation

<center><img src="./img/fixedPoint.svg" alt="fixedPoint"/></center>

This MPS fixed point can be computed through a variation on the VUMPS algorithm introduced in the previous chapter, as will be explained in the next section. Suppose for now that we have managed to find an MPS representation $\left | \Psi(A) \right \rangle$ of the fixed point of $T(O)$. The corresponding eigenvalue $\Lambda$ is then given by

$$ \Lambda = \left \langle \Psi(\bar{A}) \middle | T \middle | \Psi(A) \right \rangle , $$

assuming as always that we are dealing with a properly normalized MPS. If we bring $\left | \Psi(A) \right \rangle$ in mixed canonical form, then $\Lambda$ is given by the network

<center><img src="./img/lambda.svg" alt="lambda"/></center>

We can contract this resulting infinite network by finding the fixed points of the left and right channel operators $T_L$ and $T_R$ which are defined as

<center><img src="./img/channels.svg" alt="channels"/></center>

The corresponding fixed points $F_L$ and $F_R$, also referred to as the left and right *environments*, satisfy

<center><img src="./img/environments.svg" alt="environments"/></center>

and can be normalized such that

<center><img src="./img/envNorm.svg" alt="envNorm"/></center>

The eigenvalues $\lambda_L$ and $\lambda_R$ have to correspond to the same value $\lambda$ by construction, so that $\Lambda$ is given by

$$ \Lambda = \lim_{N \to \infty} \lambda^N ,$$

where $N$ is the number of sites in the horizontal direction. Finally, we note that we can associate a *free energy density* $f = -\frac{1}{N} \log \Lambda = -\log \lambda$ to this MPS fixed point.


### 3.2 The VUMPS algorithm for MPOs

In order to formulate an algorithm for finding this MPS fixed point we start by stating the optimality condition it must satisfy in order to qualify as an approximate eigenvector of $T(O)$. Intuitively, what we would like to impose is that the residual $T(O) \left| \Psi \right \rangle - \Lambda \left | \Psi \right \rangle$ is equal to zero. While this condition can never be satisfied exactly for any MPS approximation, we can however demand that the tangent space projection of this residual vanishes,

$$
\mathcal{P}_A \left( T(O) \left| \Psi \right \rangle - \Lambda \left | \Psi \right \rangle \right) = 0,
$$

where $\mathcal{P}_A$ is the projector onto the tangent space to the MPS manifold at $A$. Similar to the Hamiltonian case, this projected residual can be characterized in terms of a tangent space gradient $G$,

$$
G = A_C' - A_L C' = A_c' - C' A_R,
$$

where $A_C'$ and $C'$ are now given by

<center><img src="./img/AcPrime2.svg" alt="AcPrime2"/></center>

and

<center><img src="./img/CPrime2.svg" alt="CPrime2"/></center>

The optimality condition for the fixed point MPS is then equivalent to having $||G|| = 0$. In addition, if the MPO defining the transfer matrix is hermitian then it can be shown that the optimality condition corresponds to the variational minimum of the free energy density introduced above. Similar to the Hamiltonian case, if we introduce operators $O_{A_C}(\cdot)$ and $O_C(\cdot)$ such that

$$
\begin{align}
O_{A_C}(A_C) = A_C', \\
O_{C}(C) = C',
\end{align}
$$

then it follows that the fixed point is characterized by the set of equations

$$
\begin{align}
O_{A_C}(A_C) \propto A_C, \\
O_{C}(C) \propto C, \\
A_C = A_L C = C A_R.
\end{align}
$$

The VUMPS algorithm for MPOs then corresponds to an iterative scheme for finding the solutions to these equations starting from a given set $\{A_L, A_C, A_R, C\}$ which consists of the following steps:

1. Find the left and right environments $F_L$ and $F_R$ and use these to solve the eigenvalue equations for $O_{A_C}$ and $O_C$, giving new center tensors $\tilde{A}_C$ and $\tilde{C}$.

2. From these new center tensors, extract the $\tilde{A}_L$ and $\tilde{A}_R$ that minimize $||\tilde{A}_C - \tilde{A}_L \tilde{C}||$ and $||\tilde{A}_C - \tilde{C} \tilde{A}_L||$ using the `minAcC` routine from the previous chapter.

3. Update the set of tensors $\{A_L, A_C, A_R, C\} \leftarrow \{\tilde{A}_L, \tilde{A}_C, \tilde{A}_R, \tilde{C}\}$ and evaluate the optimality condition $\varepsilon = \left | \left | O_{A_C} (A_C) - A_L O_C(C) \right | \right |$.

4. If the optimality condition lies above the given tolerance, repeat.

#### Implementing the VUMPS algorithm

We start by implementing the routines for finding and normalizing the left and right environments of the channel operators. Note that, for the sake of simplicity, we will assume throughout this tutorial that the MPO tensor $O$ is a `TensorMap` of rank (4, 0) with a trivial codomain. This ensures that the index ordering of the MPO always behaves in the expected way (though this may not be the most natural partitioning in the context of MPS algorithms, see for example the [MPSKit.jl documentation](https://maartenvd.github.io/MPSKit.jl/dev/man/intro/)).

In [ ]:
"""
Compute the left environment as the fixed point of the left channel operator.

### Arguments

- `O::TensorMap{CartesianSpace, 4, 0}`: MPO tensor with 4 legs of dimension (d, d, d, d), ordered left-top-right-bottom.
- `Al::TensorMap{CartesianSpace, 2, 1}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, left orthonormal.
- `tol::Float64=1e-5`: tolerance for eigenvalue solver.

### Returns

- `lam::Float64`: left leading eigenvalue.
- `Fl::TensorMap{CartesianSpace, 2, 1}`: left environment tensor with 3 legs of dimension (D, d, D), ordered bottom-middle-top.
"""
function leftEnvironment(O, Al; tol=1e-5)
    tol = max(tol, 1e-14)
    
    # compute the largest magnitude eigenvalue and corresponding eigenvector
    Fl0 = TensorMap(randn, scalartype(Al), space(Al, 1) ⊗ space(O, 1) ← space(Al, 1))
    vals, vecs, _ = eigsolve(Fl0, 1, :LM; tol=tol) do v
        @tensor vout[-1 -2; -3] := v[5 3; 1] * Al[1 2; -3] * conj(Al[5 4; -1]) * O[3 2 -2 4]
    end
    
    return vals[1], vecs[1]
end

"""
Compute the right environment as the fixed point of the right channel operator.

### Arguments

- `O::TensorMap{CartesianSpace, 4, 0}`: MPO tensor with 4 legs of dimension (d, d, d, d), ordered left-top-right-bottom.
- `Ar::TensorMap{CartesianSpace, 1, 2}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, right orthonormal.
- `tol::Float64=1e-5`: tolerance for eigenvalue solver.

### Returns

- `lam::Float64`: right leading eigenvalue.
- `Fr::TensorMap{CartesianSpace, 2, 1}`: right environment tensor with 3 legs of dimension (D, d, D), ordered top-middle-bottom..
"""
function rightEnvironment(O, Ar; tol=1e-5)
    tol = max(tol, 1e-14)
    
    # compute the largest magnitude eigenvalue and corresponding eigenvector
    Fr0 = TensorMap(randn, scalartype(Ar), space(Ar, 3) ⊗ space(O, 3) ← space(Ar, 3))
    vals, vecs, _ = eigsolve(Fr0, 1, :LM; tol=tol) do v
        @tensor vout[-1 -2; -3] := v[1 3; 5] * Ar[-1; 2 1] * conj(Ar[-3; 4 5]) * O[-2 2 3 4]
    end
    
    return vals[1], vecs[1]
end

"""
Compute the left and right environments of the channel operators as well as the corresponding eigenvalue.

### Arguments

- `O::TensorMap{CartesianSpace, 4, 0}`: MPO tensor with 4 legs of dimension (d, d, d, d), ordered left-top-right-bottom.
- `Al::TensorMap{CartesianSpace, 2, 1}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, left orthonormal.
- `Ar::TensorMap{CartesianSpace, 1, 2}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, right orthonormal.
- `C::TensorMap{CartesianSpace, 1, 1}`: center gauge tensor with 2 legs of dimension (D, D), ordered left-right.
- `tol::Float64=1e-5`: tolerance for eigenvalue solver.

### Returns

- `lam::Float64`: right leading eigenvalue.
- `Fl::TensorMap{CartesianSpace, 2, 1}`: left environment tensor with 3 legs of dimension (D, d, D), ordered bottom-middle-top.
- `Fr::TensorMap{CartesianSpace, 2, 1}`: right environment tensor with 3 legs of dimension (D, d, D), ordered top-middle-bottom..
"""
function environments(O, Al, Ar, C; tol=1e-5)
    tol = max(tol, 1e-14)
    
    lamL, Fl = leftEnvironment(O, Al; tol)
    _, Fr = rightEnvironment(O, Ar; tol)
    
    @tensor overlap = Fl[1 3; 2] * Fr[5 3; 4] * C[2; 5] * conj(C[1; 4])
    
    return real(lamL), Fl / overlap, Fr
end;

Next we implement the action of the effective operators $O_{A_C}$ and $O_C$ on a given input tensor,

<center><img src="./img/O_Ac.svg" alt="O_Ac"/></center>

and

<center><img src="./img/O_C.svg" alt="O_C"/></center>

In [ ]:
"""
Action of the operator O_Ac on a given tensor.

### Arguments

- `x::TensorMap{CartesianSpace, 2, 1}`: tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.
- `O::TensorMap{CartesianSpace, 4, 0}`: MPO tensor with 4 legs of dimension (d, d, d, d), ordered left-top-right-bottom.
- `Fl::TensorMap{CartesianSpace, 2, 1}`: left environment tensor with 3 legs of dimension (D, d, D), ordered bottom-middle-top.
- `Fr::TensorMap{CartesianSpace, 2, 1}`: right environment tensor with 3 legs of dimension (D, d, D), ordered top-middle-bottom..
- `lam::Float64`: leading eigenvalue.

### Returns

- `y::TensorMap{CartesianSpace, 2, 1}`: result of action of O_Ac on `x`, given as a tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.
"""
function O_Ac(x, O, Fl, Fr, lam)
    
    @tensor y[-1 -2; -3] := Fl[-1 2; 1] * Fr[4 5; -3] * x[1 3; 4] * O[2 3 5 -2]
    
    return y / lam
end

"""
Action of the operator O_C on a given tensor.

### Arguments

- `x::TensorMap{CartesianSpace, 1, 1}`: tensor with 2 legs of dimension (D, D), ordered left-right.
- `Fl::TensorMap{CartesianSpace, 2, 1}`: left environment tensor with 3 legs of dimension (D, d, D), ordered bottom-middle-top.
- `Fr::TensorMap{CartesianSpace, 2, 1}`: right environment tensor with 3 legs of dimension (D, d, D), ordered top-middle-bottom..

### Returns

- `y::TensorMap{CartesianSpace, 1, 1}`: result of action of O_C on `x`, given as a tensor with 2 legs of dimension (D, D), ordered left-right.
"""
function O_C(x, Fl, Fr)
    
    @tensor y[-1; -2] := Fl[-1 3; 1] * Fr[2 3; -2] * x[1; 2]
    
    return y
end;

This then allows to define a new routine `calcNewCenterMpo` which finds the new center tensors $\tilde{A}_C$ and $\tilde{C}$ by solving the eigenvalue problems for $O_{A_C}$ and $O_C$.

In [ ]:
"""
Find new guess for Ac and C as fixed points of the maps O_Ac and O_C.

### Arguments

- `O::TensorMap{CartesianSpace, 4, 0}`: MPO tensor with 4 legs of dimension (d, d, d, d), ordered left-top-right-bottom.
- `Ac::TensorMap{CartesianSpace, 2, 1}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, center gauge.
- `C::TensorMap{CartesianSpace, 1, 1}`: center gauge tensor with 2 legs of dimension (D, D), ordered left-right.
- `Fl::TensorMap{CartesianSpace, 2, 1}`: left environment tensor with 3 legs of dimension (D, d, D), ordered bottom-middle-top.
- `Fr::TensorMap{CartesianSpace, 2, 1}`: right environment tensor with 3 legs of dimension (D, d, D), ordered top-middle-bottom..
- `lam::Float64`: leading eigenvalue.
- `tol::Float64=1e-5`: tolerance for eigenvalue solver.

### Returns

- `Ãc::TensorMap{CartesianSpace, 2, 1}`: new center gauge MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.
- `C̃::TensorMap{CartesianSpace, 1, 1}`: new center gauge tensor with 2 legs of dimension (D, D), ordered left-right.
"""
function calcNewCenterMpo(O, Ac, C, Fl, Fr, lam; tol=1e-5)
    tol = max(tol, 1e-14)
    
    # compute fixed points of effective operators as new guess for center tensors
    _, vecs = eigsolve(x -> O_Ac(x, O, Fl, Fr, lam), Ac, 1, :LM; tol)
    Ãc = vecs[1]
    _, vecs = eigsolve(x -> O_C(x, Fl, Fr), C, 1, :LM; tol)
    C̃ = vecs[1]
        
    return Ãc, C̃
end;

Since the `minAcC` routine to extract a new set of mixed gauge MPS tensors from the updated $\tilde{A}_C$ and $\tilde{C}$ can be reused from the previous chapter, we now have all the tools needed to implement the VUMPS algorithm for MPOs.

In [ ]:
"""
Find the fixed point MPS of a given MPO using VUMPS.

### Arguments

- `O::TensorMap{CartesianSpace, 4, 0}`: MPO tensor with 4 legs of dimension (d, d, d, d), ordered left-top-right-bottom.
- `D::Int`: bond dimension.
- `A0::TensorMap{CartesianSpace, 2, 1}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right.
- `tol::Float64=1e-4`: relative convergence criterium.

### Returns

- `lam::Float64`: leading eigenvalue.
- `Al::TensorMap{CartesianSpace, 2, 1}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, left orthonormal.
- `Ac::TensorMap{CartesianSpace, 2, 1}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, center gauge.
- `Ar::TensorMap{CartesianSpace, 1, 2}`: MPS tensor with 3 legs of dimension (D, d, D), ordered left-bottom-right, right orthonormal.
- `C::TensorMap{CartesianSpace, 1, 1}`: center gauge tensor with 2 legs of dimension (D, D), ordered left-right.
- `Fl::TensorMap{CartesianSpace, 2, 1}`: left environment tensor with 3 legs of dimension (D, d, D), ordered bottom-middle-top.
- `Fr::TensorMap{CartesianSpace, 2, 1}`: right environment tensor with 3 legs of dimension (D, d, D), ordered top-middle-bottom..
"""
function vumpsMpo(O, D, A0=createMPS(D, dim(space(O, 2))); tol=1e-4, tolFactor=1e-2, verbose=true)
    tol = max(tol, 1e-14)
        
    # go to mixed gauge
    Al, Ac, Ar, C = mixedCanonical(A0)
    
    delta = 1e-4
    flag = true
    i = 0

    while flag
        i += 1
        
        # compute left and right environments and corrsponding eigenvalue
        lam, Fl, Fr = environments(O, Al, Ar, C; tol=delta*tolFactor)
        
        # calculate convergence measure, check for convergence
        @tensor G[-1 -2; -3] := O_Ac(Ac, O, Fl, Fr, lam)[-1 -2; -3] - Al[-1, -2, 1] * O_C(C, Fl, Fr)[1, -3]
        delta = norm(G)

        delta < tol && (flag = false)
        
        # compute updates on Ac and C
        Ãc, C̃ = calcNewCenterMpo(O, Ac, C, Fl, Fr, lam; tol=delta*tolFactor)
        
        # find Al, Ar from Ãc, C̃
        Ãl, Ãc, Ãr, C̃ = minAcC(Ãc, C̃; tol=delta*tolFactor^2)
        
        # update tensors
        Al, Ac, Ar, C = Ãl, Ãc, Ãr, C̃
        
        # print current eigenvalue
        if verbose
            @printf "iteration:\t%d\tlambda:\t%.12f\tgradient norm:\t%.4e\n" i lam delta
        end

        if !flag
            return lam, Al, Ac, Ar, C, Fl, Fr
        end
    end
end;

### 3.3 The two-dimensional classical Ising model

Next we apply the VUMPS algorithm for MPOs to the two-dimensional classical Ising model. To this end, consider classical spins $s_i = \pm 1$ placed on the sites of an infinite square lattice which interact according to the nearest-neighbor Hamiltonian

$$H = -J \sum_{\langle i,j \rangle} s_i s_j \,.$$

We now wish to compute the corresponding partition function,

$$ \mathcal{Z} = \sum_{\{s_i\}} \text{e}^{-\beta H({\{s_i\}})},$$

using our freshly implemented algorithm. In order to do this we first rewrite this partition function as the contraction of an infinite two-dimensional tensor network,

<center><img src="./img/Z.svg" alt="Z"/></center>

where every edge in the network has bond dimension $d = 2$. Here, the black dots represent $ \delta $-tensors

<center><img src="./img/delta.svg" alt="delta"/></center>

and the matrices $t$ encode the Boltzmann weights associated to each nearest-neighbor interaction,

<center><img src="./img/t.svg" alt="t"/></center>

In order to arrive at a translation invariant network corresponding to a single hermitian MPO tensor we can take the matrix square root $q$ of each Boltzmann matrix such that

<center><img src="./img/q.svg" alt="q"/></center>

and absorb the result symmetrically into the $\delta$-tensors at each vertex to define the MPO tensor

<center><img src="./img/O.svg" alt="O"/></center>

The partition function then becomes

<center><img src="./img/Z2.svg" alt="Z2p"/></center>

which precisely corresponds to an infinite power of a row-to-row transfer matrix $T(O)$ of the kind defined above. We can therefore use the VUMPS algorithm to determine its fixed point, where the corresponding eigenvalue automatically gives us the free energy density as explained before.

Having found this fixed point and its corresponding environments, we can easily evaluate expectation values of local observables. For example, say we want to find the expectation value of the magnetization at site $\mu$,

$$ \langle m \rangle = \frac{1}{\mathcal{Z}} \sum_{\{s_i\}} s_\mu \text{e}^{-\beta H({\{s_i\}})}.$$

We can access this quantity by introducing a magnetization tensor $M$, placing it at site $\mu$ and contracting the partition function network around it as

<center><img src="./img/Mexp.svg" alt="Mexp"/></center>

where the normalization factor $\mathcal{Z}$ in the denominator is taken care of by the same contraction where $O$ is left at site $\mu$ (which in this case is of course nothing more than the eigenvalue $\lambda$). The magnetization tensor $M$ is defined entirely analogously to the MPO tensor $O$, but where instead of a regular $\delta$-tensor the entry $i=j=k=l=1$ (using base-0 indexing) is set to $-1$ instead of $1$.

We can now define the routines for constructing the Ising MPO and magnetization tensor an computing local expectation values, as well as a routine that implements [Onsager's exact solution for this model](https://en.wikipedia.org/wiki/Ising_model#Onsager's_exact_solution) to compare our results to.

In [ ]:
"""
Gives the MPO tensor corresponding to the partition function of the 2d 
classical Ising model at a given temperature and coupling, obtained by
distributing the Boltzmann weights evenly over all vertices.

### Arguments

- `beta::Float64`: inverse temperature.
- `J::Float64`: coupling strength.

### Returns

- `O::TensorMap{CartesianSpace, 4, 0}`: MPO tensor with 4 legs of dimension (2, 2, 2, 2), ordered left-top-right-bottom.
"""
function isingO(beta, J)
    # basic vertex delta tensor
    vertex = zeros(ComplexF64, 2, 2, 2, 2)
    [vertex[i, i, i, i] = 1 for i in 1:2]
    vertex = Tensor(vertex, ℝ^2 ⊗ ℝ^2 ⊗ ℝ^2 ⊗ ℝ^2)
    # take matrix square root of Boltzmann weights and pull into vertex edges
    t = ComplexF64[exp(beta*J) exp(-beta*J); exp(-beta*J) exp(beta*J)]
    t = TensorMap(t, ℝ^2 ← ℝ^2)
    q = sqrt(t)
    @tensor O[-1 -2 -3 -4] := q[-1; 1] * q[-2; 2] * q[-3; 3] * q[-4; 4] * vertex[1 2 3 4]
    return O
end

"""
Gives the magnetizatopn MPO tensor for the 2d classical Ising model at a
given temperature and coupling.

### Arguments

- `beta::Float64`: inverse temperature.
- `J::Float64`: coupling strength.

### Returns

- `M::TensorMap{CartesianSpace, 4, 0}`: magnetization tensor with 4 legs of dimension (2, 2, 2, 2), ordered left-top-right-bottom.
"""
function isingM(beta, J)
    # almost basic vertex delta tensor
    vertex = zeros(ComplexF64, 2, 2, 2, 2)
    vertex[1, 1, 1, 1] = 1
    # with opposite sign on (2, 2, 2, 2) entry
    vertex[2, 2, 2, 2] = -1
    vertex = Tensor(vertex, ℝ^2 ⊗ ℝ^2 ⊗ ℝ^2 ⊗ ℝ^2)
    # take matrix square root of Boltzmann weights and pull into vertex edges
    t = ComplexF64[exp(beta*J) exp(-beta*J); exp(-beta*J) exp(beta*J)]
    t = TensorMap(t, ℝ^2 ← ℝ^2)
    q = sqrt(t)
    @tensor M[-1 -2 -3 -4] := q[-1; 1] * q[-2; 2] * q[-3; 3] * q[-4; 4] * vertex[1 2 3 4]
    return M
end

"""
Gives the expectation value of a local operator O.

### Arguments

- `O::TensorMap{CartesianSpace, 4, 0}`: local operator of which we want to compute the expectation value, with 4 legs of timension (d, d, d, d),ordered left-top-right-bottom.
- `Ac::TensorMap{CartesianSpace, 2, 1}`: MPS tensor of the MPS fixed point, with 3 legs of dimension ordered left-bottom-right, center gauge.
- `Fl::TensorMap{CartesianSpace, 2, 1}`: left environment tensor with 3 legs of dimension (D, d, D), ordered bottom-middle-top.
- `Fr::TensorMap{CartesianSpace, 2, 1}`: right environment tensor with 3 legs of dimension (D, d, D), ordered top-middle-bottom..
### Returns

- `e::Float64`: expectation value of the operator O.
"""
function expValMpo(O, Ac, Fl, Fr)
    @tensor e = Fl[1 3; 2] * Ac[2 7; 5] * O[3 7 8 6] * conj(Ac[1 6; 4]) * Fr[5 8; 4]
    
    return e
end

"""
Exact Onsager solution for the 2d classical Ising Model.

### Arguments

- `beta::Float64`: inverse temperature.
- `J::Float64`: coupling strength.

### Returns
- `magnetization::Float64`: magnetization at given temperature and coupling.
- `free::Float64`: free energy at given temperature and coupling.
- `energy::Float64`: energy at given temperature and coupling.
"""
function isingExact(beta, J)
    theta = 0:1e-6:pi/2
    x = 2 * sinh(2 * J * beta) / cosh(2 * J * beta)^2
    if 1 - (sinh(2 * J * beta))^(-4) > 0
        magnetization = (1 - (sinh(2 * J * beta))^(-4))^(1 / 8)
    else
        magnetization = 0
    end
    free = -1 / beta * (log(2 * cosh(2 * J * beta)) + 1 / pi * trapz(theta, log.(1 / 2 * (1 .+ sqrt.(1 .- x^2 * sin.(theta).^2)))))
    K = trapz(theta, 1 ./ sqrt.(1 .- x^2 * sin.(theta).^2))
    energy = -J * cosh(2 * J * beta) / sinh(2 * J * beta) * (1 + 2 / pi * (2 * tanh(2 * J * beta)^2 - 1) * K)
    return magnetization, free, energy
end;

We can now demonstrate the VUMPS algorithm for MPOs. We will fix $J = 1$ in the following, and investigate the behavior of the model as a function of temperature. Since we know the critical piont is located at $T_c = \frac{2}{\log\left(1 + \sqrt{2}\right)} \approx 2.26919$, let us first have a look at $T = 4$ and $T = 1$ far above and below the critical temperature, for which we expect a vanishing and non-vanishing magnetization respectively.

In [ ]:
D = 12
d = 2
J = 1
tol = 1e-8
tolFactor = 1e-4
A0 = createMPS(D, d)

for T in [4, 1]
    println("Running for T = $(T)\n")
    beta = 1 / T
    O = isingO(beta, J)
    M = isingM(beta, J)
    lam, Al, Ac, Ar, C, Fl, Fr = vumpsMpo(O, D, A0; tol, tolFactor, verbose=true)
    mag = abs(expValMpo(M, Ac, Fl, Fr) / expValMpo(O, Ac, Fl, Fr))
    freeEnergy = -log(lam)/beta
    magExact, freeEnergyExact, _ = isingExact(beta, J)
    @printf("\nFree energy: %.10f; \trelative difference with exact solution: %.4e\n\n",
                    freeEnergy, abs((freeEnergy - freeEnergyExact) / freeEnergyExact))
    @printf("Magnetization: %.10f\n\n", mag)
end

We clearly see that far from the critical point the VUMPS algorithm achieves excellent agreement with the exact solution efficiently at very small bond dimensions.

As a final demonstration, we compute the magnetization and free energy over a range from $T = 1$ to $T = 3.4$ and plot the results. Note that convergence of the algorithm slows down significantly near the critical point, as can be expected.

In [ ]:
D = 12
d = 2
J = 1

println("Bond dimension: D = $(D)\n")
Al = createMPS(D, d)
# optimization parameters
tol = 1e-8
tolFactor = 1e-2
verbose = false

Ts = LinRange(1., 3.4, 100)
magnetizations = []
magnetizationsExact = []
freeEnergies = []
freeEnergiesExact = []

for T in Ts
    beta = 1 / T
    O = isingO(beta, J)
    M = isingM(beta, J)
    flush(stdout)
    @printf "Running for T = %.4f \r" T
    lam, Al, Ac, Ar, C, Fl, Fr = vumpsMpo(O, D, Al; tol, tolFactor, verbose)
    magEx, freeEx, _ = isingExact(beta, J)
    push!(magnetizations, abs(expValMpo(M, Ac, Fl, Fr)/expValMpo(O, Ac, Fl, Fr)))
    push!(magnetizationsExact, magEx)
    push!(freeEnergies, -log(lam)/beta)
    push!(freeEnergiesExact, freeEx)
end

p1 = scatter(
    Ts,
    magnetizations;
    label="D = $(D)",
    title="Magnetization as a function of the temperature",
    xlabel=L"T",
    ylabel=L"\langle M \rangle",
    marker=:utriangle,
    color="hotpink",
)
plot!(Ts, magnetizationsExact; label="exact")
p2 = scatter(
    Ts,
    freeEnergies;
    label="D = $(D)",
    title="Free energy as a function of the temperature",
    xlabel=L"T",
    ylabel=L"f",
    marker=:utriangle,
    color="hotpink"
)
plot!(Ts, freeEnergiesExact; label="exact")
plot(p1, p2, layout=(1, 2), size=(1200, 350), margin=5Plots.mm)